## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도 경제성장을 이룩한 선진국가이다. 전통 문화와 현대 문명이 공존하며 혼합된 문화적 특성을 지니고 있다. 또한 K-pop, K-drama, K-beauty 등의 문화 콘텐츠로 세계적으로 큰 인기를 끌고 있으며, 경제적으로는 세계 10대 경제국 중 하나이며, 기술 혁신 분야에서도 선두를 달리고 있다. 또한 남북 간의 관계가 여전히 긴장 상태이며 이를 해결하기 위한 노력을 지속하고 있다. 한류 열풍과 함께 세계적으로 알려진 나라로, 문화, 경제, 기술 등 다양한 분야에서 높은 발전을 이루고 있는 국가이다.
CPU times: user 40.7 ms, sys: 25.7 ms, total: 66.4 ms
Wall time: 2.97 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [24]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고대 문명 국가로, 역사와 전통을 간직한 독특한 문화를 가지고 있다. 현재는 세계적으로 발전한 IT 기술을 보유한 선진국가로 발전해 있으며, K-pop, K-drama 등을 통해 대중문화 붐을 일으키고 있다. 또한 한국은 고부지, 한류, 한식 등 다양한 측면으로 세계인들에게 많은 영향을 주고 있으며, 아름다운 자연경관과 높은 품질의 생활환경도 자랑할 만하다. 한반도 북한과의 관계는 여전히 긴장 상태에 있지만, 남북 관계의 평화와 통일을 위해 노력하는 모습도 보여주고 있다.
CPU times: user 1.88 ms, sys: 780 μs, total: 2.66 ms
Wall time: 2.02 ms


In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

## SQLite Cache


In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [13]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고대 문명 국가로, 역사와 전통을 간직한 독특한 문화를 가지고 있다. 현재는 세계적으로 발전한 IT 기술을 보유한 선진국가로 발전해 있으며, K-pop, K-drama 등을 통해 대중문화 붐을 일으키고 있다. 또한 한국은 고부지, 한류, 한식 등 다양한 측면으로 세계인들에게 많은 영향을 주고 있으며, 아름다운 자연경관과 높은 품질의 생활환경도 자랑할 만하다. 한반도 북한과의 관계는 여전히 긴장 상태에 있지만, 남북 관계의 평화와 통일을 위해 노력하는 모습도 보여주고 있다.
CPU times: user 3.1 ms, sys: 1.53 ms, total: 4.63 ms
Wall time: 4.44 ms
